### QUESTÃO 1)

In [263]:
# Carregando arquivo CSV

import csv

def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [x for x in dataset[i]]
    return dataset

datSet = loadCsv('carData.csv')
# print("Numero de tuplas: {0}".format(len(datSet)))

#print(datSet)

# Transformar categorizado em numerico

def catToNum(dataset):
    catList = []
    
    for i in range(len(dataset)):
        lTup = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        catList.append(lTup)
    
    for i in range(len(dataset)):
        for j in range(len(dataset[0])):
            if dataset[i][j] == 'low' or dataset[i][j] == 'small' or dataset[i][j] == 'unacc' or dataset[i][j] == 2:
                catList[i][j] = 1.0
            elif dataset[i][j] == 'med' or dataset[i][j] == 'acc' or dataset[i][j] == 3:
                catList[i][j] = 2.0
            elif dataset[i][j] == 'high' or dataset[i][j] == 'big' or dataset[i][j] == 'good' or dataset[i][j] == 4:
                catList[i][j] = 3.0
            elif dataset[i][j] == 'vhigh' or dataset[i][j] == 'vgood' or dataset[i][j] == 5:
                catList[i][j] = 4.0
            elif dataset[i][j] == 'more':
                catList[i][j] = 5.0
            elif dataset[i][j] == '5more':
                catList[i][j] = 6.0
    return catList

numDatSet = catToNum(datSet)
# print(numDatSet)

# Dividindo dados em treino e teste

import random

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

splitSet = splitDataset(numDatSet, 0.50) # por algum motivo a acc chega quase nos 70 quando fica meio a meio :v
trainSet = splitSet[0]
testSet = splitSet[1]

# SUMARIZAÇAO: Separar classes

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

# SUMARIZAÇAO: Calculo da media e do desvio padrao

import math

def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

# SUMARIZAÇAO: Sumarizando dados

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

# SUMARIZAÇAO: Sumarizando atributos por classe

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

sumData = summarizeByClass(trainSet)

# PREDIÇAO: Densidade da probabilidade gaussiana

def calculateProbability(x, mean, stdev):
    if(stdev == 0):
        stdev = 10000
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

#PREDIÇAO: Probabilidade das classes

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

#PREDIÇAO: Prediçao :v

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

# PREDIÇAO: Multiplas prediçoes :v

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

predData = getPredictions(sumData, testSet)
# print(predData)

# PREDÇAO: Acuracia

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

acc = getAccuracy(testSet, predData)
print(acc)

70.48611111111111


### QUESTÃO 2)

In [255]:
from sklearn.naive_bayes import GaussianNB

gauss = GaussianNB()
trainX = []
trainY = []
for i in trainSet:
    trainX.append(i[0:5])
    trainY.append(i[-1])
# print(trainX)
# print(trainY)
gauss.fit(trainX, trainY)

testX = []
testY = []
for i in testSet:
    testX.append(i[0:5])
    testY.append(i[-1])
gauss.score(testX, testY)

0.6701388888888888

### QUESTÃO 3)

Com o scrikit conseguimos manter a acurácia fixa, enquanto com o Naive Bayes feito na mão ele tende a variar entre 64~70 %.